### LLM Approach for CIT MIT

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

### Dummy Data

In [2]:
import random
import pandas as pd
from datetime import datetime, timedelta

# Parameters
n_cards = 10000
n_transactions_target = 100000
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)

# Country codes
countries = ["USA", "CAN", "IND", "AUS", "BRA", "DEU", "FRA", "JPN", "MEX", "ZAF"]

# Assign each card a home country
card_home_country = {f"card_{i}": random.choice(countries) for i in range(1, n_cards + 1)}

# Merchant categories & merchants
merchant_categories = {
    "Streaming": ["Netflix", "Hulu", "Disney+", "Spotify", "Amazon Prime"],
    "Utilities": ["NYC Electricity", "Tokyo Water", "Sydney Gas", "Berlin Power", "Mumbai Electricity"],
    "CreditCard": ["Amex Bill", "Visa Payment", "Mastercard Payment"],
    "Groceries": ["Walmart", "Carrefour", "Tesco", "Reliance Fresh", "Costco"],
    "Ecommerce": ["Amazon", "Flipkart", "eBay", "Rakuten", "MercadoLibre"],
    "Dining": ["McDonalds", "Starbucks", "Subway", "Dominos", "KFC"],
    "Travel": ["Delta Airlines", "Air India", "Qantas", "Lufthansa", "Japan Airlines"],
    "Healthcare": ["CVS Pharmacy", "Apollo Hospitals", "Walgreens", "HealthPlus"],
    "Insurance": ["StateFarm", "Allianz", "LIC", "Axa"],
    "Other": ["Ikea", "Best Buy", "Target", "GameStop", "Apple Store"]
}

# Helper: random date
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

# Generate recurring dates for given months
def recurring_dates(start, end, day, freq="M"):
    dates = []
    current = start.replace(day=day)
    while current <= end:
        dates.append(current.strftime("%Y-%m-%d"))
        if freq == "M":  # monthly
            month = current.month + 1
            year = current.year
            if month > 12:
                month = 1
                year += 1
            try:
                current = current.replace(year=year, month=month, day=day)
            except ValueError:
                # Handle shorter months (like Feb 30)
                current = current.replace(year=year, month=month, day=28)
        elif freq == "Q":  # quarterly
            month = current.month + 3
            year = current.year
            if month > 12:
                month -= 12
                year += 1
            try:
                current = current.replace(year=year, month=month, day=day)
            except ValueError:
                current = current.replace(year=year, month=month, day=28)
    return dates

# Create dataset
data = []

for card in card_home_country:
    home_country = card_home_country[card]

    # Subscriptions (10% chance per card)
    if random.random() < 0.1:
        merchant = random.choice(merchant_categories["Streaming"])
        amount = random.choice([9.99, 12.99, 15.99])
        day = random.randint(1, 28)  # safe day for all months
        for d in recurring_dates(start_date, end_date, day, freq="M"):
            country = home_country if random.random() > 0.02 else random.choice([c for c in countries if c != home_country])
            data.append([card, country, amount, "Streaming", merchant, d])

    # Bills (30% chance per card)
    if random.random() < 0.3:
        category = random.choice(["Utilities", "CreditCard", "Insurance"])
        merchant = random.choice(merchant_categories[category])
        base = random.randint(30, 200)
        day = random.randint(1, 28)
        freq = "M" if random.random() < 0.8 else "Q"
        for d in recurring_dates(start_date, end_date, day, freq=freq):
            amount = round(base * random.uniform(0.8, 1.2), 2)
            country = home_country if random.random() > 0.02 else random.choice([c for c in countries if c != home_country])
            data.append([card, country, amount, category, merchant, d])

    # Ad-hoc (random ~ 5–15 transactions per card)
    for _ in range(random.randint(5, 15)):
        category = random.choice(["Groceries", "Ecommerce", "Dining", "Travel", "Healthcare", "Other"])
        merchant = random.choice(merchant_categories[category])
        amount = round(random.uniform(5, 500), 2)
        d = random_date(start_date, end_date).strftime("%Y-%m-%d")
        country = home_country if random.random() > 0.02 else random.choice([c for c in countries if c != home_country])
        data.append([card, country, amount, category, merchant, d])

# Convert to DataFrame
df = pd.DataFrame(data, columns=["card_number", "country_code", "transaction_amount", 
                                 "merchant_category", "merchant_name", "transaction_date"])

# Downsample if exceeding target transactions
if len(df) > n_transactions_target:
    df = df.sample(n_transactions_target, random_state=42).reset_index(drop=True)

print(df.head())
print("Total transactions:", len(df))
print("Unique cards:", df['card_number'].nunique())



df.to_parquet("transactions_dummy.parquet", index=False)

  card_number country_code  transaction_amount merchant_category  \
0   card_2341          DEU               65.26        CreditCard   
1   card_4768          MEX              280.88            Travel   
2   card_5509          USA              164.20             Other   
3   card_1107          IND              223.75         Groceries   
4    card_835          CAN              225.84            Dining   

    merchant_name transaction_date  
0       Amex Bill       2024-04-07  
1  Delta Airlines       2024-06-28  
2          Target       2024-11-18  
3           Tesco       2024-01-28  
4       McDonalds       2024-12-26  
Total transactions: 100000
Unique cards: 9997


In [52]:
df.groupby('merchant_category').size().to_frame('num_rows').sort_values('num_rows',ascending=False)

,num_rows
merchant_category,
Groceries,11672
Healthcare,11626
Other,11625
Ecommerce,11600
Travel,11548
Dining,11473
Streaming,8130
Utilities,7712
Insurance,7372


In [50]:
df[df['merchant_name'].str.contains('Reliance',na=False)]

,card_number,country_code,transaction_amount,merchant_category,merchant_name,transaction_date
3,card_7611,FRA,467.38,Groceries,Reliance Fresh,2024-07-02
41,card_7332,USA,447.66,Groceries,Reliance Fresh,2024-04-22
76,card_4525,DEU,70.98,Groceries,Reliance Fresh,2024-10-01
87,card_4913,FRA,401.03,Groceries,Reliance Fresh,2024-10-07
131,card_608,DEU,452.84,Groceries,Reliance Fresh,2024-05-18
...,...,...,...,...,...,...
99746,card_9333,IND,446.34,Groceries,Reliance Fresh,2024-12-09
99761,card_8081,JPN,279.01,Groceries,Reliance Fresh,2024-10-10
99823,card_9126,FRA,393.17,Groceries,Reliance Fresh,2024-07-10
99844,card_5216,IND,93.25,Groceries,Reliance Fresh,2024-11-15


In [42]:
df[(df['merchant_category'] == 'Groceries') & (df['country_code'] == 'USA') & (df['card_number'] == 'card_10')].sort_values(['card_number','transaction_date']).to_dict(orient='records')

[{'card_number': 'card_10',
  'country_code': 'USA',
  'transaction_amount': 378.7,
  'merchant_category': 'Groceries',
  'merchant_name': 'Walmart',
  'transaction_date': '2024-01-03'},
 {'card_number': 'card_10',
  'country_code': 'USA',
  'transaction_amount': 148.01,
  'merchant_category': 'Groceries',
  'merchant_name': 'Walmart',
  'transaction_date': '2024-03-15'}]

In [13]:
df.groupby(['merchant_category','card_number']).size().to_frame('num_rows').sort_values('num_rows',ascending=False).head(4)

,,num_rows
merchant_category,card_number,
Travel,card_1960,7
Ecommerce,card_109,7
Groceries,card_8209,7
Utilities,card_3305,7


## Train Test Split

In [11]:
gpd = df.groupby(['card_number','country_code','merchant_category','merchant_name'],as_index=False)['transaction_date'].max()
gpd = gpd.rename(columns={'transaction_date':'max_transaction_date'})
train_test_data = df.merge(gpd, on = ['card_number','country_code','merchant_category','merchant_name'], how = 'left')
train_data = train_test_data[train_test_data['transaction_date'] < train_test_data['max_transaction_date']]
test_data = train_test_data[train_test_data['transaction_date'] >= train_test_data['max_transaction_date']]
train_data.shape, test_data.shape

((33205, 7), (66795, 7))

In [12]:
from transformers import pipeline

In [34]:
# Define categories dynamically in the prompt
candidate_labels = ["M103", "M102", "M101"]

# Classify each transaction
for _, row in test_data.iterrows():
    card = row['card_number']
    country = row['country_code']
    mcc = row['merchant_category']
    mname = row['merchant_name']
    context = train_data[(train_data['card_number'] == card) & (train_data['country_code'] == country) & 
                        (train_data['merchant_category'] == mcc) & (train_data['merchant_name'] == mname)]
    if context.shape[0] > 0:
        context_json = context.to_dict(orient='records')
        print(context_json)
        break


[{'card_number': 'card_2720', 'country_code': 'FRA', 'transaction_amount': 53.95, 'merchant_category': 'Utilities', 'merchant_name': 'Mumbai Electricity', 'transaction_date': '2024-09-01', 'max_transaction_date': '2024-12-01'}, {'card_number': 'card_2720', 'country_code': 'FRA', 'transaction_amount': 53.83, 'merchant_category': 'Utilities', 'merchant_name': 'Mumbai Electricity', 'transaction_date': '2024-03-01', 'max_transaction_date': '2024-12-01'}, {'card_number': 'card_2720', 'country_code': 'FRA', 'transaction_amount': 46.71, 'merchant_category': 'Utilities', 'merchant_name': 'Mumbai Electricity', 'transaction_date': '2024-08-01', 'max_transaction_date': '2024-12-01'}, {'card_number': 'card_2720', 'country_code': 'FRA', 'transaction_amount': 43.98, 'merchant_category': 'Utilities', 'merchant_name': 'Mumbai Electricity', 'transaction_date': '2024-02-01', 'max_transaction_date': '2024-12-01'}, {'card_number': 'card_2720', 'country_code': 'FRA', 'transaction_amount': 51.9, 'merchant_c

In [101]:
%%time

def get_prompt(input_txn, context):
    # Provide context examples
    prompt = f"""
    You are a data analyst. Your task is to look at each input transaction, have a look at the transaction history of that card, 
    in the same country, same merchant and same merchant category. After doing that, classify each transaction into: M103, M102, M101. 
    You can use the following reasoning to do so.
    
    Chain-of-Thought: Look for repeating patterns in transaction amount around the same date of a month from previous transactions. 
    If such a pattern exists, we can be sure that it is M103. Try using elimination method, if no pattern emerges showing it is M103, 
    check if a pattern for M102 can be seen. The definition for M102 is as follows:
    If the transaction's amount is varying but the transaction date is within a week difference of last few months' transaction date 
    then we can categorize it as M102.
    If there is no context or history of transactions, classify it as M101. Otherwise classify the transaction as M101.
    
    Example 1:
    Input:
    {{
      "card_number": "card_1101",
      "country_code": "USA",
      "transaction_amount": 15.99,
      "merchant_category": "Streaming",
      "merchant_name": "Netflix",
      "transaction_date": "2024-12-19"
    }}
    
    Context:
    [
      {{
        "card_number": "card_1101",
        "country_code": "USA",
        "transaction_amount": 15.99,
        "merchant_category": "Streaming",
        "merchant_name": "Netflix",
        "transaction_date": "2024-01-19"
      }},
      {{
        "card_number": "card_1101",
        "country_code": "USA",
        "transaction_amount": 15.99,
        "merchant_category": "Streaming",
        "merchant_name": "Netflix",
        "transaction_date": "2024-02-19"
      }},
      {{
        "card_number": "card_1101",
        "country_code": "USA",
        "transaction_amount": 15.99,
        "merchant_category": "Streaming",
        "merchant_name": "Netflix",
        "transaction_date": "2024-05-19"
      }}
    ]
    Category: M103
    Reason: The transaction is repeating almost every month around the same date with the same amount for the same number.
    
    Example 2:
    Input:
    {{
      "card_number": "card_10",
      "country_code": "USA",
      "transaction_amount": 148.01,
      "merchant_category": "Groceries",
      "merchant_name": "Walmart",
      "transaction_date": "2024-03-15"
    }}
    
    Context:
    [
      {{
        "card_number": "card_10",
        "country_code": "USA",
        "transaction_amount": 378.7,
        "merchant_category": "Groceries",
        "merchant_name": "Walmart",
        "transaction_date": "2024-01-03"
      }}
    ]
    Category: M101
    Reason: The transaction doesn't show any repeating pattern neither through transaction amount nor through transaction date.
    
    Example 3:
    Input:
    {{
      "card_number": "card_2720",
      "country_code": "FRA",
      "transaction_amount": 43.68,
      "merchant_category": "Utilities",
      "merchant_name": "Mumbai Electricity",
      "transaction_date": "2024-12-01"
    }}
    
    Context:
    [
      {{
        "card_number": "card_2720",
        "country_code": "FRA",
        "transaction_amount": 43.98,
        "merchant_category": "Utilities",
        "merchant_name": "Mumbai Electricity",
        "transaction_date": "2024-02-01"
      }},
      {{
        "card_number": "card_2720",
        "country_code": "FRA",
        "transaction_amount": 53.83,
        "merchant_category": "Utilities",
        "merchant_name": "Mumbai Electricity",
        "transaction_date": "2024-03-01"
      }},
      {{
        "card_number": "card_2720",
        "country_code": "FRA",
        "transaction_amount": 52.3,
        "merchant_category": "Utilities",
        "merchant_name": "Mumbai Electricity",
        "transaction_date": "2024-07-01"
      }},
      {{
        "card_number": "card_2720",
        "country_code": "FRA",
        "transaction_amount": 46.71,
        "merchant_category": "Utilities",
        "merchant_name": "Mumbai Electricity",
        "transaction_date": "2024-08-01"
      }}
    ]
    Category: M102
    Reason: The transaction shows a varying monthly/quarterly bill pattern.
    
    Now, please use this information to classify the incoming transaction:
    
    Input:
    {input_txn},
    
    Context:
    {context}
    """

    return prompt

for idx, row in test_data.head(5).iterrows():
    print(f"For index {idx}")
    card = row['card_number']
    country = row['country_code']
    mcc = row['merchant_category']
    mname = row['merchant_name']
    txndate = row['transaction_date']
    txnamt = row['transaction_amount']
    context = train_data[(train_data['card_number'] == card) & (train_data['country_code'] == country) & 
                        (train_data['merchant_category'] == mcc) & (train_data['merchant_name'] == mname)]
    input_txn = test_data[(test_data['card_number'] == card) & (test_data['country_code'] == country) & 
                        (test_data['merchant_category'] == mcc) & (test_data['merchant_name'] == mname)].to_dict(orient='records')
    context_json = context.to_dict(orient='records')
    # print(context_json)
    prompt = get_prompt(input_txn, context_json)
    print("Starting classification!")
    # result = classifier(prompt)
    result = classifier(prompt, labels)
    # print(result)
    print("Final Category",result['scores'])


For index 1
Starting classification!
Final Category [0.5071039795875549, 0.30860835313796997, 0.18428772687911987]
For index 3
Starting classification!
Final Category [0.5071039795875549, 0.30860835313796997, 0.18428772687911987]
For index 6
Starting classification!
Final Category [0.5071039795875549, 0.30860835313796997, 0.18428772687911987]
For index 10
Starting classification!
Final Category [0.5071039795875549, 0.30860835313796997, 0.18428772687911987]
For index 11
Starting classification!
Final Category [0.5071039795875549, 0.30860835313796997, 0.18428772687911987]
CPU times: user 356 ms, sys: 189 ms, total: 545 ms
Wall time: 3.43 s


In [85]:
%%time
from transformers import pipeline

# Load instruction-tuned model
# model_name = "google/flan-t5-large"
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# classifier = pipeline("text2text-generation", model=model_name)

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use mps:0


CPU times: user 52.4 s, sys: 10 s, total: 1min 2s
Wall time: 2min 28s


In [3]:
from langchain import PromptTemplate

In [5]:
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path = '/Users/vishaldawar/Phi-3-mini-4k-instruct-fp16.gguf',
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx = 2048,
    seed=42,
    verbose=False
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (n

In [66]:
# template = """You are a data analyst. Answer the following questions as best you can. You have access to the following tools:

# {tools}

# Use the following format:

# Question: the input question you must answer
# Information: you should always extract insights from the information provided below and use that logic to apply on the question asked by looking through the context provided
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Information provided:

# Your task is to look at each input transaction, have a look at the transaction history of that card, 
# in the same country, same merchant and same merchant category. After doing that, classify each transaction into: M103, M102, M101. 
# You can use the following reasoning to do so.

# Chain-of-Thought: Look for repeating patterns in transaction amount around the same date of a month from previous transactions. 
# If such a pattern exists, we can be sure that it is M103. Try using elimination method, if no pattern emerges showing it is M103, 
# check if a pattern for M102 can be seen. The definition for M102 is as follows:
# If the transaction's amount is varying but the transaction date is within a week difference of last few months' transaction date 
# then we can categorize it as M102.
# If there is no context or history of transactions, classify it as M101. Otherwise classify the transaction as M101.

# Example 1:
# Input:
# {{
#   "card_number": "card_1101",
#   "country_code": "USA",
#   "transaction_amount": 15.99,
#   "merchant_category": "Streaming",
#   "merchant_name": "Netflix",
#   "transaction_date": "2024-12-19"
# }}

# Context:
# [
#   {{
#     "card_number": "card_1101",
#     "country_code": "USA",
#     "transaction_amount": 15.99,
#     "merchant_category": "Streaming",
#     "merchant_name": "Netflix",
#     "transaction_date": "2024-01-19"
#   }},
#   {{
#     "card_number": "card_1101",
#     "country_code": "USA",
#     "transaction_amount": 15.99,
#     "merchant_category": "Streaming",
#     "merchant_name": "Netflix",
#     "transaction_date": "2024-02-19"
#   }},
#   {{
#     "card_number": "card_1101",
#     "country_code": "USA",
#     "transaction_amount": 15.99,
#     "merchant_category": "Streaming",
#     "merchant_name": "Netflix",
#     "transaction_date": "2024-05-19"
#   }}
# ]
# Category: M103
# Reason: The transaction is repeating almost every month around the same date with the same amount.

# Example 2:
# Input:
# {{
#   "card_number": "card_10",
#   "country_code": "USA",
#   "transaction_amount": 148.01,
#   "merchant_category": "Groceries",
#   "merchant_name": "Walmart",
#   "transaction_date": "2024-03-15"
# }}

# Context:
# [
#   {{
#     "card_number": "card_10",
#     "country_code": "USA",
#     "transaction_amount": 378.7,
#     "merchant_category": "Groceries",
#     "merchant_name": "Walmart",
#     "transaction_date": "2024-01-03"
#   }}
# ]
# Category: M101
# Reason: The transaction doesn't show any repeating pattern neither through transaction amount nor through transaction date.

# Example 3:
# Input:
# {{
#   "card_number": "card_2720",
#   "country_code": "FRA",
#   "transaction_amount": 43.68,
#   "merchant_category": "Utilities",
#   "merchant_name": "Mumbai Electricity",
#   "transaction_date": "2024-12-01"
# }}

# Context:
# [
#   {{
#     "card_number": "card_2720",
#     "country_code": "FRA",
#     "transaction_amount": 43.98,
#     "merchant_category": "Utilities",
#     "merchant_name": "Mumbai Electricity",
#     "transaction_date": "2024-02-01"
#   }},
#   {{
#     "card_number": "card_2720",
#     "country_code": "FRA",
#     "transaction_amount": 53.83,
#     "merchant_category": "Utilities",
#     "merchant_name": "Mumbai Electricity",
#     "transaction_date": "2024-03-01"
#   }},
#   {{
#     "card_number": "card_2720",
#     "country_code": "FRA",
#     "transaction_amount": 52.3,
#     "merchant_category": "Utilities",
#     "merchant_name": "Mumbai Electricity",
#     "transaction_date": "2024-07-01"
#   }},
#   {{
#     "card_number": "card_2720",
#     "country_code": "FRA",
#     "transaction_amount": 46.71,
#     "merchant_category": "Utilities",
#     "merchant_name": "Mumbai Electricity",
#     "transaction_date": "2024-08-01"
#   }}
# ]
# Category: M102
# Reason: The transaction shows a varying monthly/quarterly bill pattern.

# Question: {input}.

# Input:
# {input_txn},

# Context:
# {context}

# Thought:{agent_scratchpad} 
# """

template = """|user|
You are a data analyst. Your task is to look at each input transaction, have a look at the transaction history of that card, 
in the same country, same merchant and same merchant category. After doing that, classify each transaction into: M103, M102, M101. 
You can use the following reasoning to do so.

Chain-of-Thought: Look for repeating patterns in transaction amount around the same date of a month from previous transactions. 
If such a pattern exists, we can be sure that it is M103. Try using elimination method, if no pattern emerges showing it is M103, 
check if a pattern for M102 can be seen. The definition for M102 is as follows:
If the transaction's amount is varying but the transaction date is within a week difference of last few months' transaction date 
then we can categorize it as M102.
If there is no context or history of transactions, classify it as M101. Otherwise classify the transaction as M101.

Example 1:
Input:
{{
  "card_number": "card_1101",
  "country_code": "USA",
  "transaction_amount": 15.99,
  "merchant_category": "Streaming",
  "merchant_name": "Netflix",
  "transaction_date": "2024-12-19"
}}

Context:
[
  {{
    "card_number": "card_1101",
    "country_code": "USA",
    "transaction_amount": 15.99,
    "merchant_category": "Streaming",
    "merchant_name": "Netflix",
    "transaction_date": "2024-01-19"
  }},
  {{
    "card_number": "card_1101",
    "country_code": "USA",
    "transaction_amount": 15.99,
    "merchant_category": "Streaming",
    "merchant_name": "Netflix",
    "transaction_date": "2024-02-19"
  }},
  {{
    "card_number": "card_1101",
    "country_code": "USA",
    "transaction_amount": 15.99,
    "merchant_category": "Streaming",
    "merchant_name": "Netflix",
    "transaction_date": "2024-05-19"
  }}
]
Category: M103
Reason: The transaction is repeating almost every month around the same date with the same amount for the same number.

Example 2:
Input:
{{
  "card_number": "card_10",
  "country_code": "USA",
  "transaction_amount": 148.01,
  "merchant_category": "Groceries",
  "merchant_name": "Walmart",
  "transaction_date": "2024-03-15"
}}

Context:
[
  {{
    "card_number": "card_10",
    "country_code": "USA",
    "transaction_amount": 378.7,
    "merchant_category": "Groceries",
    "merchant_name": "Walmart",
    "transaction_date": "2024-01-03"
  }}
]
Category: M101
Reason: The transaction doesn't show any repeating pattern neither through transaction amount nor through transaction date.

Example 3:
Input:
{{
  "card_number": "card_2720",
  "country_code": "FRA",
  "transaction_amount": 43.68,
  "merchant_category": "Utilities",
  "merchant_name": "Mumbai Electricity",
  "transaction_date": "2024-12-01"
}}

Context:
[
  {{
    "card_number": "card_2720",
    "country_code": "FRA",
    "transaction_amount": 43.98,
    "merchant_category": "Utilities",
    "merchant_name": "Mumbai Electricity",
    "transaction_date": "2024-02-01"
  }},
  {{
    "card_number": "card_2720",
    "country_code": "FRA",
    "transaction_amount": 53.83,
    "merchant_category": "Utilities",
    "merchant_name": "Mumbai Electricity",
    "transaction_date": "2024-03-01"
  }},
  {{
    "card_number": "card_2720",
    "country_code": "FRA",
    "transaction_amount": 52.3,
    "merchant_category": "Utilities",
    "merchant_name": "Mumbai Electricity",
    "transaction_date": "2024-07-01"
  }},
  {{
    "card_number": "card_2720",
    "country_code": "FRA",
    "transaction_amount": 46.71,
    "merchant_category": "Utilities",
    "merchant_name": "Mumbai Electricity",
    "transaction_date": "2024-08-01"
  }}
]
Category: M102
Reason: The transaction shows a varying monthly/quarterly bill pattern.

Question: Please use this information to classify the incoming transaction as M103, M102 or M101.

Input:
{input_txn},

Context:
{context}
<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template = template,
    input_variables = ['input_txn', 'context']
)

basic_chain = prompt | llm

In [68]:
%%time
for idx, row in test_data.head(5).iterrows():
    print(f"For index {idx}")
    card = row['card_number']
    country = row['country_code']
    mcc = row['merchant_category']
    mname = row['merchant_name']
    txndate = row['transaction_date']
    txnamt = row['transaction_amount']
    context = train_data[(train_data['card_number'] == card) & (train_data['country_code'] == country) & 
                        (train_data['merchant_category'] == mcc) & (train_data['merchant_name'] == mname)]
    input_txn = test_data[(test_data['card_number'] == card) & (test_data['country_code'] == country) & 
                        (test_data['merchant_category'] == mcc) & (test_data['merchant_name'] == mname)].to_dict(orient='records')
    context_json = context.to_dict(orient='records')
    print("Number of txns in context ",len(context))
    val = basic_chain.invoke({
        "input" : "Please use this information to classify the incoming transaction as M103, M102 or M101?",
        "input_txn" : input_txn,
        "context" : context_json
    })
    print(val)


For index 1
Number of txns in context  0
 Category: M102
Reason: The transaction shows a varying monthly bill pattern. Although there is only one transaction in the context, we can assume that this card has previous transactions related to travel expenses (Delta Airlines) around similar dates, indicating it could be classified as an annual travel subscription with varying amounts due to different services or destinations each time. Without more data points for clear M103 classification, and considering there are no specific patterns in the single transaction provided, M102 is a reasonable category based on the given logic and reasoning.
For index 2
Number of txns in context  0
 Category: M101
Reason: There is no context or history of transactions, so we cannot classify it based on repeating patterns, varying amounts, or dates. The input transaction stands alone without any reference to previous similar transactions for the same card number in the USA for 'Other' merchant category at Ta

In [69]:
26975

for idx, row in test_data[test_data.index==26975].iterrows():
    print(f"For index {idx}")
    card = row['card_number']
    country = row['country_code']
    mcc = row['merchant_category']
    mname = row['merchant_name']
    txndate = row['transaction_date']
    txnamt = row['transaction_amount']
    context = train_data[(train_data['card_number'] == card) & (train_data['country_code'] == country) & 
                        (train_data['merchant_category'] == mcc) & (train_data['merchant_name'] == mname)]
    if context.shape[0] > 5:
        context = context.sort_values('transaction_date').head(5)
    input_txn = test_data[(test_data['card_number'] == card) & (test_data['country_code'] == country) & 
                        (test_data['merchant_category'] == mcc) & (test_data['merchant_name'] == mname)].to_dict(orient='records')
    context_json = context.to_dict(orient='records')
    val = basic_chain.invoke({
        "input" : "Please use this information to classify the incoming transaction as M103, M102 or M101?",
        "input_txn" : input_txn,
        "context" : context_json
    })
    print(val)

For index 26975
 Category: M103
Reason: The transaction is repeating almost every month around the same date with the same amount for the same number.


## Agents

In [47]:
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

prompt = PromptTemplate(
    template= react_template,
    input_variables=["tools","tool_names", "input", "agent_scratchpad"]
)

In [18]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

In [48]:
# search = DuckDuckGoSearchResults()
search = DDGS()

search_tool = Tool(
    name='duckduck',
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.text,
)

tools = load_tools(['llm-math'], llm=llm)
tools.append(search_tool)

In [27]:
from ddgs import DDGS

In [49]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools = tools, verbose=True, handle_parsing_errors = True
)

In [51]:
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD."
    }
)



> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action:: 
<|assistant|> Question: What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD.
Thought: I need to find out the current price first and then calculate its equivalent in EUR using the given exchange rate.
Action: duckduck
Action Input: "current price of a MacBook Pro"

Assuming the search result gives us $1200 as the current price for a MacBook Pro, I'll use this value to perform my next calculation.

Thought: Now that I have the price in USD, I can calculate its equivalent in EUR using the given exchange rate of 0.85 EUR/USD.
Action: Calculator
Action Input: "1200 * 0.85"

Final Answer: The current price of a MacBook Pro is $1200, and it would cost approximately €1020 in EUR if the exchange rate is 0.85 EUR for 1 USD.

(Note: Since we are using hypothetical values here, please replace "$1200" with the act

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD.',
 'output': 'Based on the hypothetical value of $1200 for a MacBook Pro, it would cost approximately €1020 in EUR at an exchange rate of 0.85 EUR per USD. Please note that you need to replace "$1200" with the actual price found from an online search.\n\n(Note: The final answer provided here is based on a hypothetical value and should be updated with real-time data when performing this task.)'}